In [5]:
import pandas as pd
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
train_vision = pd.read_csv('train_vision.csv')

In [3]:

datagen = ImageDataGenerator( 
    channel_shift_range = 0.1,
        zoom_range=0.3,
    brightness_range=[0.7, 1.3],
        fill_mode='nearest')

for name, label in train_vision.values:
    img = load_img('faces_images/'+name)  # PIL 이미지
    x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
    x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

    # 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
    # 지정된 `preview/` 폴더에 저장합니다.
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir='faces_images', save_prefix=name[:-4] + '_aug', save_format='png'):
        i += 1
        if i > 1:
            break  # 이미지 1장을 생성하고 마칩니다


In [10]:
image_list = np.array(os.listdir('faces_images'))
image_df = pd.DataFrame(image_list, columns = ['filename'])

In [11]:
image_df.describe()

,filename
count,19699
unique,19699
top,face_5827.png
freq,1


In [12]:
image_df[image_df['filename'] == '.DS_Store']

,filename
2823,.DS_Store


In [13]:
image_df = image_df.drop(index=2823, axis = 0)

In [14]:
image_df['label'] = 0

In [15]:
train_vision['name'] = train_vision['filename'].str.extract('([a-z\_0-9]+)')

In [16]:
image_df['name'] = image_df['filename'].str.extract('([a-z\_0-9]{6,9})')
image_df['name'] = image_df['name'].str.rstrip('u')
image_df['name'] = image_df['name'].str.rstrip('a')
image_df['name'] = image_df['name'].str.rstrip('_')

In [18]:
for i in range(len(train_vision.values)):
    name = train_vision.loc[i, 'name']; la = train_vision.loc[i, 'label']
    image_df['label'][image_df['name'] == name] = la

/home/pirl/anaconda3/envs/vnect/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
del_index = image_df[(image_df['label'] == 0)].index

In [21]:
image_df = image_df.drop(index = del_index, axis = 0)

In [25]:
image_df = image_df.drop(columns=['name'])

In [26]:
image_df

,filename,label
0,face_5669.png,2
1,face_7196_aug_0_6055.png,4
2,face_606_aug_0_5096.png,4
5,face_6490_aug_0_1678.png,1
6,face_3612_aug_0_3699.png,4
...,...,...
19694,face_7406_aug_0_394.png,4
19695,face_4661_aug_0_3584.png,1
19696,face_3430.png,1
19697,face_5090.png,4


In [27]:
image_df.to_csv('train_vision_aug_1.csv', index = False)

In [23]:
5850*2

11700

In [24]:
5850*3

17550